In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from QuantLib import *

plt.style.use('fivethirtyeight')

In [17]:
start_date = Date(20, 4, 2018)
maturity_date = Date(20, 7, 2018)
freq = Period(1, Days)
option_type = Option.Put
strike_price = 1.
spot_price = 1.

risk_free_rate = 0.035
dividend_rate = 0.035
day_count = Actual365Fixed()
calendar = China(China.SSE)

payoff = PlainVanillaPayoff(option_type, strike_price)

calculation_date = start_date
Settings.instance().evaluationDate = calculation_date

spot_handle = RelinkableQuoteHandle(SimpleQuote(spot_price))
flat_ts = YieldTermStructureHandle(FlatForward(calculation_date, risk_free_rate, day_count))
dividend_yield = YieldTermStructureHandle(FlatForward(calculation_date, dividend_rate, day_count))
flat_vol_ts = RelinkableBlackVolTermStructureHandle(BlackConstantVol(calculation_date, calendar, 0.2, day_count))

In [18]:
maturities = [Date(21, 5, 2018), Date(20, 6, 2018), Date(20, 7, 2018)]
fixing_start_dates = [Date(20, 4, 2018), Date(21, 5, 2018), Date(20, 6, 2018)]
vols = [0.33, 0.34, 0.35]

## Forward Asian Options Groups
-------------------------

In [33]:
print(f"Forward Asian Option evaluated at {calculation_date.ISO()}\n")

prices = []

for fix_start, maturity, vol in zip(fixing_start_dates, maturities, vols):
    fixing_scheduler = Schedule(fix_start, maturity, freq, calendar, Following, Following, DateGeneration.Backward, False)
    fixing_dates = [d for d in fixing_scheduler]
    flat_vol_ts.linkTo(BlackConstantVol(calculation_date, calendar, vol, day_count))
    
    bsm_process = BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)
    exercise = EuropeanExercise(maturity)
    option = DiscreteAveragingAsianOption(Average.Arithmetic, 0., 0, fixing_dates, payoff, exercise)
    engine = FdBlackScholesAsianEngine(bsm_process, 200, 100, 100)
    option.setPricingEngine(engine)
    price = option.NPV()
    delta = option.delta()
    print(f"{fix_start.ISO()} - {maturity.ISO()}, PV: {price:.4f};  Δ: {delta:.4f}")
    prices.append(price)

Forward Asian Option evaluated at 2018-04-20

2018-04-20 - 2018-05-21, PV: 0.0225;  Δ: -0.4899
2018-05-21 - 2018-06-20, PV: 0.0445;  Δ: -0.4752
2018-06-20 - 2018-07-20, PV: 0.0610;  Δ: -0.4654


In [34]:
prices

[0.02251000579243813, 0.04453160690826882, 0.060978203833682854]

In [42]:
df = pd.DataFrame({'结算日': [d.ISO() for d in maturities],
                   '均价起始': [d.ISO() for d in fixing_start_dates],
                   '计算日': [calculation_date.ISO()] * len(maturities)})
df['执行价格'] = 16000
df['期权单价'] = df['执行价格'] * np.array(prices)
df[['计算日', '均价起始', '结算日', '执行价格', '期权单价']]

,计算日,均价起始,结算日,执行价格,期权单价
0,2018-04-20,2018-04-20,2018-05-21,16000,360.160093
1,2018-04-20,2018-05-21,2018-06-20,16000,712.505711
2,2018-04-20,2018-06-20,2018-07-20,16000,975.651261
